In [73]:
import yfinance as yf
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from lxml import etree
from datetime import datetime
from dateutil.relativedelta import relativedelta
import requests
import pandas as pd
import numpy as np
import datetime
import json
import threading
import asyncio
from asyncio_throttle import Throttler
import aiohttp
import tracemalloc
import time

params = {
  'access_key': 'fc86254346a54a4fa6f26c220adbfab8'
}

header = {
    'user-agent': 'Mozilla/5.0'
}

# yahoo finance overriding the pandas datareader
yf.pdr_override()


## sp500 stocks 
all_tickers_xpath = '//span[text() = "S&P 500 component stocks"]/following::tbody[1]/tr/td[1]/a'
soup = BeautifulSoup(requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies', headers=header).content, 'html.parser')
dom = etree.HTML(str(soup))

# find all tickers
all_tickers = [i.text for i in dom.xpath(all_tickers_xpath)]

<h3>Getting data for all sp500 stocks based on a certain start date</h3>

In [74]:
# def yfinance_get_data(tickers: list[str], start: datetime = datetime(2022, 1, 3)):
#     return pdr.get_data_yahoo(' '.join(tickers), start=start)

# data = yfinance_get_data(all_tickers, start = datetime(2021, 1, 1))

<h5>Reading CSV data of stock market and formatting</h5>

In [75]:
data = pd.read_csv('stockdata.csv')
above = [i.split('.')[0] for i in data.columns][1:]
below = data.iloc[0][1:]
data.set_index('Unnamed: 0', inplace=True)
data.columns = pd.MultiIndex.from_arrays([above, below])
data = data.drop('Date').rename_axis('Date')
data = data.iloc[1:,:]
data = data.stack().reset_index().set_index('Date').rename(columns={0 : 'Ticker'})

# it would be nice to have index as timestamp objects rather than
# just strings
def convert_to_datetime(s):
    s = s.split('-')
    yr = int(s[0])
    mo = int(s[1])
    day = int(s[2].split(' ')[0])

    return datetime.datetime(yr, mo, day)

data.index = list(map(convert_to_datetime, data.index))
data = data.rename_axis('Date')
data.head()

C:\Users\abhin\AppData\Local\Temp\ipykernel_6316\2506647805.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255

,Ticker,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
2021-01-04,A,117.03971099853516,118.63999938964844,120.08999633789062,117.11000061035156,118.94000244140625,2030700
2021-01-04,AAL,15.130000114440918,15.130000114440918,15.880000114440918,15.029999732971191,15.850000381469727,69732500
2021-01-04,AAP,147.68507385253906,157.33999633789062,159.57000732421875,155.9600067138672,157.89999389648438,849100
2021-01-04,AAPL,127.67998504638672,129.41000366210938,133.61000061035156,126.76000213623047,133.52000427246094,143301900
2021-01-04,ABBV,95.02763366699219,105.41000366210938,107.3499984741211,103.86000061035156,107.18000030517578,9523400


<h4>Unfortunately, 500 stock indicators is too much data, so we have to shorten this list to a smaller subset of stocks to study. We will focus on the top 25 stocks of SP500.</h4>

In [76]:
top_stocks = '''AAPL MSFT AMZN NVDA GOOGL BRK.A GOOG TSLA META XOM UNH JNJ JPM V PG MA CVX HD ABBV LLY MRK AVGO PEP KO PFE'''
top_stocks = top_stocks.split(' ')

# subset the dataset to the rows that contain the stock symbols
data_subset = data[data['Ticker'].isin(top_stocks)]

In [77]:
grouped = data_subset.groupby('Ticker')
# this example group will just be the sample dataset that
# we may do calculations with 
example_group = grouped.get_group(top_stocks[0])

In [78]:
# finanicial indicator functions
# note that these are indented to work with ONE key value
# ex. ONLY DATES means that we must group by stock ticker
# note: TICKERS will not make sense for these functions as keys

def check_series_numeric(column) -> bool:
    not_numeric = column.apply(pd.to_numeric, errors = 'coerce').isna().all()
    # if the data is not numeric, the test failed 
    return not not_numeric

def calculate_sma(column, period = 10):
    if not check_series_numeric(column):
        return column
    sma = column.rolling(period).mean()
    return pd.Series(sma, index= column.index, name = 'SMA')

def calculate_ema(column, period = 10):
    if not check_series_numeric(column):
        return column
    ema = column.ewm(span = period, min_periods = period - 1).mean()
    return pd.Series(ema, index = column.index, name = 'EMA')

# formula for RSI: 
'''
    let RS = average gain / average loss   
    RSI = 100 - 100 / (1 - RS)

    some things to look out for: 
        - using exponential weighted means on the average
            gains and losses made the RSI value much more
            sensitive and fluctuating more based on price
            changes
        - using simple averages made the RSI value more 
            smoothed out

    keyword argument options:
        - period: type int
        - use_exponential: type bool

'''
def calculate_rsi(column, *args, **kwargs):
    rsi = column.rolling(14).apply(lambda x: calculate_rsi_helper(x, *args, **kwargs)).astype(float)
    return pd.Series(rsi, index = column.index, name = 'RSI')

def calculate_rsi_helper(column, period = 14, use_exponential = False):
    if not check_series_numeric(column):
        return column 

    column = column.astype(float)
    # find differences in prices
    differences = column.diff() 

    # clip, but center off of zero value
    gain = differences.clip(lower = 0.01)
    loss = differences.clip(upper = -0.01)

    avg_gain = gain.mean()
    avg_loss = loss.mean()

    if use_exponential:
        # get the exponential weighted mean of the very last element in this current rolling window
        avg_gain = gain.ewm(span = period, min_periods = period - 1).mean()[-1]
        avg_loss = loss.ewm(span = period, min_periods = period - 1).mean()[-1]

    RS = avg_gain / avg_loss 
    RSI = 100 - 100 / (1 - RS)

    return RSI

'''
    MACD (moving average convergence/ divergence) shows the relationship
    between two exponential moving averages and comparing this to the 
    9-day EMA line 

    MACD = 12-period EMA - 26-period EMA 
    Signal = 9-period EMA - 26-period EMA
'''
def calculate_macd(column, long_period = 26, short_period = 12, signal_period = 9) -> tuple[object, object]:
    if not check_series_numeric(column):
        return column 
    
    ewm_short = example_group['Adj Close'].ewm(span = short_period, min_periods = 11).mean()
    ewm_long = example_group['Adj Close'].ewm(span = long_period, min_periods = 25).mean()

    # returns the macd 
    macd = (ewm_short - ewm_long).astype(float)

    # returns the signal line
    ewm_signal = example_group['Adj Close'].ewm(span = signal_period, min_periods = 8).mean()
    signal = (ewm_signal - ewm_long).astype(float)


    macd = pd.Series(macd, index = column.index, name = 'MACD')
    signal = pd.Series(signal, index = column.index, name = 'MACD-SIGNAL')
    return macd, signal


In [79]:
# TODO: apply this process concurrently or parallel if possible, then we can get
# even more data available to us
dataframes = []
for ticker in top_stocks:
    cur = None
    try:
        cur = grouped.get_group(ticker)
    except:
        print('skipped ticker:', ticker)
        continue 

    stock_data = cur['Adj Close']
    sma = calculate_sma(stock_data)
    ema = calculate_ema(stock_data)
    rsi = calculate_rsi(stock_data)
    macd, signal_macd = calculate_macd(stock_data)

    overall = pd.DataFrame([sma, ema, rsi, macd, signal_macd]).T
    res = pd.merge(cur, overall, on='Date')

    res = res.set_index([res.index, res['Ticker']]).drop('Ticker', axis = 1)

    dataframes.append(res)
    


skipped ticker: BRK.A


In [80]:
data = pd.concat(dataframes, axis = 0)

In [81]:
data

,,Adj Close,Close,High,Low,Open,Volume,SMA,EMA,RSI,MACD,MACD-SIGNAL
Date,Ticker,,,,,,,,,,,
2021-01-04,AAPL,127.67998504638672,129.41000366210938,133.61000061035156,126.76000213623047,133.52000427246094,143301900,NaN,NaN,NaN,NaN,NaN
2021-01-05,AAPL,129.2586212158203,131.00999450683594,131.74000549316406,128.42999267578125,128.88999938964844,97664900,NaN,NaN,NaN,NaN,NaN
2021-01-06,AAPL,124.90755462646484,126.5999984741211,131.0500030517578,126.37999725341797,127.72000122070312,155088000,NaN,NaN,NaN,NaN,NaN
2021-01-07,AAPL,129.16983032226562,130.9199981689453,131.6300048828125,127.86000061035156,128.36000061035156,109578200,NaN,NaN,NaN,NaN,NaN
2021-01-08,AAPL,130.2847137451172,132.0500030517578,132.6300048828125,130.22999572753906,132.42999267578125,105158200,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-14,PFE,41.189999,41.189999,41.709999,40.93,41.509998,20613300,41.365,41.255713,61.868660,2.980509,3.526012
2023-04-17,PFE,41.18,41.18,41.279999,40.959999,41.119999,17608600,41.403,41.241947,65.507987,2.969431,3.527161
2023-04-18,PFE,40.549999,40.549999,41.330002,40.529999,41.279999,18541500,41.323,41.116139,53.527950,3.025830,3.631906


In [9]:
# creating the csv file for stock price predictions
# data.to_csv('../predictions/stock_data_all.csv')

<h3>Using Pushshift API to get Sentiment Data from Reddit</h3>
<h3>NOT DOING THIS ANYMORE: rate limits make it very hard to get all the data that is needed</h3>

<ol>
    <li>Each entry of stocks are associated with the current day and the ticker symbol</li>
    <li>We may search through top comments or posts through Pushshift to search for stock ticker mentions per day and stock</li>
    <li>In order to load numerous http requests, we are using asyncio for concurrently loading data</li>
</ol>

In [240]:
# function to convert date time string into epoch format for pushshift
def str_index_to_epoch(s: str, days_after = 1) -> tuple[int, int]:
    s = str(s)
    s = s.split('-')
    yr = int(s[0])
    mo = int(s[1])
    day = int(s[2].split(' ')[0])
    # the yahoo finance data is given as EST time
    today = datetime.datetime(yr, mo, day)
    tomorrow = today + relativedelta(days = days_after)

    print('today', today, 'tomorrow', tomorrow, 'after', days_after)
    return (int(today.timestamp()), int(tomorrow.timestamp()))

def index_str_to_datetime(s: str, split='-'):
    # get anything that is before a space, to isolate the current search thingy
    s = s.split(' ')[0]
    vals = [int(i) for i in s.split(split)]

    return datetime.datetime(year=vals[0], month=vals[1], day=vals[2])

In [341]:
def get_link(X, subreddit = 'stocks', size = 25, days_after = 1):
    subreddit = subreddit
    ticker = X['Ticker']
    date_after, date_before = str_index_to_epoch(X.name, days_after=days_after)
    size = size

    req_string_reddit_comments = f'https://api.pushshift.io/reddit/search/comment/?q={ticker}&subreddit={subreddit}&size={size}&after={date_after}&before={date_before}&fields=body&frequency=day'
    
    return req_string_reddit_comments

def create_query_link(argument_dict, type = 'comment'):
    args = []
    for argument in argument_dict:
        args.append(argument)
        args.append('=')
        args.append(argument_dict[argument])
        args.append('&')

        query_string = ''.join(args[:-1])
    return f'https://api.pushshift.io/reddit/{type}/search/?{query_string}'


In [342]:
args = {
    'q': 'AAPL',
    'subreddit': 'stocks',
    'sort': 'created_utc',
    'sort_type': 'created_utc',
    
}

create_query_link(args)

'https://api.pushshift.io/reddit/comment/search/?q=AAPL&subreddit=stocks&sort=created_utc&sort_type=created_utc'

we can simply use the api to get all date ranges for the stock market in one link

In [371]:
# using ticker aapl as the current symbol
stock_data = data.query('Ticker == "AAPL"').reset_index().set_index('Date')
start_date, end_date = stock_data.iloc[0].name, stock_data.iloc[-1].name
start_date, end_date = index_str_to_datetime(str(start_date)), index_str_to_datetime(str(end_date))

period = (end_date - start_date).days
print(period, 'days to search for')

args = {
    'q': 'AAPL',
    'subreddit': 'stocks',
    'after': str(int(start_date.timestamp())),
    'before': str(int((end_date.timestamp()))),
    'size': str(500),
    'count': str(50),
    'sort':'created_utc',
    'sort_type': 'asc'
}

link = create_query_link(args)
print(link)

836 days to search for
https://api.pushshift.io/reddit/comment/search/?q=AAPL&subreddit=stocks&after=1609740000&before=1681966800&size=500&count=50&sort=created_utc&sort_type=asc


In [412]:
dataframes = []

def roll_apply(window):
    print(window)

counter = 1
for i in range(0, len(stock_data), 30):
    window = stock_data.index[i:i + 60]
    after = index_str_to_datetime(str(window[0])).timestamp()
    before = index_str_to_datetime(str(window[-1])).timestamp()

    args = {
    'q': 'AAPL',
    'subreddit': 'stocks',
    'after': str(int(after)),
    'before': str(int(before)),
    'size': str(500),
    'count': str(50),
    'sort':'created_utc',
    'sort_type': 'asc'
    }

    print('request #', counter)

    r = requests.get(create_query_link(args))
    result = json.loads(r.text)['data']
    dataframes.append(pd.DataFrame(result))

    counter += 1

    

request # 1
request # 2
request # 3
request # 4
request # 5
request # 6
request # 7
request # 8
request # 9
request # 10
request # 11
request # 12
request # 13
request # 14
request # 15
request # 16
request # 17
request # 18
request # 19
request # 20


In [413]:
sentiment_data = pd.concat(dataframes, axis = 0)
sentiment_data = sentiment_data[['body', 'utc_datetime_str']]
sentiment_data['date'] = [index_str_to_datetime(s) for s in sentiment_data['utc_datetime_str']]
sentiment_data['date'] = [convert_to_datetime(str(s)) for s in sentiment_data['date']]

sentiment_data

,body,utc_datetime_str,date
0,Picked up 30 more AAPL and 10 more SQ today :),2021-03-30 04:06:18,2021-03-30
1,I think we're all overestimating Elon's true i...,2021-03-30 03:50:48,2021-03-30
2,"Yes it has, for small caps with people trying ...",2021-03-30 02:49:07,2021-03-30
3,oh well.\n\npile back into AAPL\n\n&amp;#x200B...,2021-03-30 01:18:31,2021-03-30
4,I’m 18 and started 5 months ago and right now ...,2021-03-30 00:28:38,2021-03-30
...,...,...,...
94,I don’t mean market consensus “undeniable winn...,2023-04-11 13:48:42,2023-04-11
95,I had 6680 shares of Waste Management in 2012 ...,2023-04-11 13:17:25,2023-04-11
96,"Portfolio 1: \nCSPX, VXUS \nAAPL, ABNB, AZN,...",2023-04-11 12:24:42,2023-04-11
97,AAPL. Strong buybacks and good dividend backed...,2023-04-11 11:21:45,2023-04-11


In [414]:
len(np.unique(sentiment_data['date']))

474

In [372]:
r = requests.get(link)

In [373]:
r.text

'{"data":[{"subreddit_id":"t5_2qjfk","author_is_blocked":false,"comment_type":null,"edited":false,"author_flair_type":"text","total_awards_received":0,"subreddit":"stocks","author_flair_template_id":null,"id":"jgy3s9j","gilded":0,"archived":false,"collapsed_reason_code":null,"no_follow":true,"author":"redditoranna","send_replies":true,"parent_id":null,"score":1,"author_fullname":"t2_1fefym7e","all_awardings":[],"body":"So, for those of you who want to buy if/when the market drops, how are you spreading it out? How many quarters? Do you have a strategy? \\n\\nI just invested a bit in the recent dip, but want to buy more in the next dips, if my desired stocks drop (AAPL, MSFT, GOOGL, AMZN, NVDA, some others). I don’t want to drop it all in during this earnings season, but don’t know how far out to plan. I know you can’t perfectly time the market, but I’m looking for sale prices, and if they don’t happen, I’m okay with what I have so far. I’m 52 and planning to hold 20-30 years.\\n\\nI’m 

In [374]:
df = pd.DataFrame(json.loads(r.text)['data'])
dates = [d.split(' ')[0] for d in df['utc_datetime_str']]

In [380]:
df['dates'] = dates

df.groupby('dates')['dates'].count()

dates
2023-02-02    19
2023-02-03    25
2023-02-04     7
2023-02-05     5
2023-02-06     4
              ..
2023-04-15     2
2023-04-16    42
2023-04-17    11
2023-04-18     2
2023-04-19     7
Name: dates, Length: 76, dtype: int64

In [382]:
len(np.unique(dates))

76

<h5>Idea: we have a lot of days to get links from, but is it possible to cover multiple days with the api call? That would be much easier</h5>

In [167]:
# limiting the number of requests to 60 per minute
semaphore = asyncio.Semaphore(2)

async def get_data(link, session, semaphore):
    data = None 

    async with semaphore:
        try:
            async with session.get(link, timeout = 5) as response:
                try:
                    data = await response.json() 
                except aiohttp.ContentTypeError:
                    print('content type error:', link) 

                return data 
        except TimeoutError:
            print('timeout error at:', link)
            
    return data 

async with aiohttp.ClientSession() as session:
    all_tasks = []
    for link in links:
        all_tasks.append(asyncio.ensure_future(get_data(link, session, semaphore)))
        asyncio.sleep(0.5)

    global result
    result = await asyncio.gather(*all_tasks)


C:\Users\abhin\AppData\Local\Temp\ipykernel_6316\4244045032.py:25: RuntimeWarning: coroutine 'sleep' was never awaited
  asyncio.sleep(0.5)


content type error: https://api.pushshift.io/reddit/search/comment/?q=AAPL&subreddit=stocks&size=50&after=1661403600&before=1662699600&fields=body&sort=asc
content type error: https://api.pushshift.io/reddit/search/comment/?q=AAPL&subreddit=stocks&size=50&after=1663304400&before=1664600400&fields=body&sort=asc
content type error: https://api.pushshift.io/reddit/search/comment/?q=AAPL&subreddit=stocks&size=50&after=1665118800&before=1666414800&fields=body&sort=asc
content type error: https://api.pushshift.io/reddit/search/comment/?q=AAPL&subreddit=stocks&size=50&after=1666933200&before=1668232800&fields=body&sort=asc
content type error: https://api.pushshift.io/reddit/search/comment/?q=AAPL&subreddit=stocks&size=50&after=1668751200&before=1670047200&fields=body&sort=asc
content type error: https://api.pushshift.io/reddit/search/comment/?q=AAPL&subreddit=stocks&size=50&after=1672812000&before=1674108000&fields=body&sort=asc
content type error: https://api.pushshift.io/reddit/search/comme